## Accuracy Plots

In [14]:
from plotting import plot_classification_accuracies, plot_segmentation_accuracies
import pandas as pd
import altair_saver
import os

repbrep = '../../repbrep/'
outdir = 'figure_out'

segmentation_predictions = pd.read_parquet(os.path.join(repbrep, 'results/segmentation_predictions.parquet'))
classification_predictions = pd.read_parquet(os.path.join(repbrep, 'results/classification_predictions.parquet'))

f360_macro = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'macro')
f360_macro_linear = plot_segmentation_accuracies(segmentation_predictions, 'Fusion360Seg', 'macro', 'linear')
f360_micro = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'micro')
f360_micro_linear = plot_segmentation_accuracies(segmentation_predictions,  'Fusion360Seg', 'micro', 'linear')

mfcad_macro = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'macro')
mfcad_macro_linear = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'macro', 'linear')
mfcad_micro = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'micro')
mfcad_micro_linear = plot_segmentation_accuracies(segmentation_predictions,  'MFCAD', 'micro', 'linear')

fabwave = plot_classification_accuracies(classification_predictions, 'FabWave')
fabwave_linear = plot_classification_accuracies(classification_predictions, 'FabWave', 'Ours', 'linear')

os.makedirs(outdir, exist_ok=True)
altair_saver.save(f360_macro, os.path.join(outdir, 'f360_macro.pdf'))
altair_saver.save(f360_macro_linear, os.path.join(outdir, 'f360_macro_linear.pdf'))
altair_saver.save(mfcad_macro, os.path.join(outdir, 'mfcad_macro.pdf'))
altair_saver.save(mfcad_macro_linear, os.path.join(outdir, 'mfcad_macro_linear.pdf'))

altair_saver.save(f360_micro, os.path.join(outdir, 'f360_micro.pdf'))
altair_saver.save(f360_micro_linear, os.path.join(outdir, 'f360_micro_linear.pdf'))
altair_saver.save(mfcad_micro, os.path.join(outdir, 'mfcad_micro.pdf'))
altair_saver.save(mfcad_micro_linear, os.path.join(outdir, 'mfcad_micro_linear.pdf'))

altair_saver.save(fabwave, os.path.join(outdir, 'fabwave.pdf'))
altair_saver.save(fabwave_linear, os.path.join(outdir, 'fabwave.pdf'))

## Reconstruction Gallery

In [ ]:
import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm.notebook import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from pspy import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os

repbrep = '../../repbrep/'
datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join('figure_out', 'reconstructions.png')

rows = 6
cols = 4
size = 5

grid_density = 100

force_rendering = True



testing_losses = torch.load(render_losses_path)
avg_losses = np.array([x[0].item() for x in testing_losses])
part_sizes = np.array([x[1] for x in testing_losses])
total_losses = avg_losses * part_sizes
avg_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][0])
total_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][2])
total_filtered = [x for x in total_sorted if x[1][1] > 20]


recompute_renderings = True
if os.path.exists('render_results.pt') and not force_rendering:
    render_results = torch.load('render_results.pt')
    renders = render_results['renders']
    gts = render_results['gts']
    if len(renders) >= rows*cols:
        recompute_renderings = False

if recompute_renderings:
    with open(f360seg_index_path,'r') as f:
        index = json.load(f)
    data =  ZipFile(f360seg_zip_path,'r')
    parts_list = [index['template'].format(*x) for x in index['test']]

    model = BRepFaceAutoencoder(64,1024,4)
    ckpt = torch.load(model_checkpoint_path)
    model.load_state_dict(ckpt['state_dict'])

    def predict(face_codes, model, N=grid_density):
        n_faces = face_codes.shape[0]
        line = torch.linspace(-0.1,1.1,N)
        grid = torch.cartesian_prod(line, line)
        grids = grid.repeat(n_faces,1)
        indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)
        with torch.no_grad():
            indexed_codes = face_codes[indices]
            uv_codes = torch.cat([grids, indexed_codes],dim=1)
            preds = model.decoder(uv_codes)
        return preds

    codes = torch.load(computed_f360seg_codes_path)

    gts = []
    renders = []
    for k in tqdm(range(rows*cols)):
        i = total_filtered[k][0]
        N = grid_density
        preds = predict(codes[parts_list[i]]['x'], model, N)
        V, F, C = preds_to_mesh(preds, N)
        part = Part(data.open(parts_list[i]).read().decode('utf-8'))
        pose, zoom = find_best_angle_from_part(part)
        ground_truth = render_part(part, pose, zoom)
        rendering = render_mesh(V,F,C,pose,zoom)
        gts.append(ground_truth)
        renders.append(rendering)

    render_results = {'gts':gts,'renders':renders}
    torch.save(render_results, 'render_results.pt')



M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols*2, figsize=(2*cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,2*col].imshow(gts[i])
    axes[row,2*col].axis('off')
    axes[row,2*col+1].imshow(renders[i])
    axes[row,2*col+1].axis('off')
fig.savefig(figure_out_path)

In [19]:
import torch
import numpy as np
from rendering import find_best_angle_from_part, render_part, render_mesh
from tqdm.notebook import tqdm
from render_shape import preds_to_mesh
from zipfile import ZipFile
import json
from pspy import Part
from train_latent_space import BRepFaceAutoencoder
from matplotlib import pyplot as plt
import os

repbrep = '../../repbrep/'
datasets = os.path.join(repbrep, 'datasets')
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
results_dir = os.path.join(repbrep, 'results')

f360seg_index_path = os.path.join(datasets, 'fusion360seg.json')
f360seg_zip_path = os.path.join(datasets, 'fusion360seg.zip')
model_checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')
computed_f360seg_codes_path = os.path.join(model_dir, 'fusion360seg_coded.pt')
render_losses_path = os.path.join(results_dir, 'f360_render_test_losses.pt')
figure_out_path = os.path.join('figure_out', 'reconstructions.png')

test_poses_path = os.path.join(datasets, 'f360seg_test_poses.npy')
test_zooms_path = os.path.join(datasets, 'f360seg_test_zooms.npy')

rows = 6
cols = 4
size = 5

grid_density = 100

force_rendering = True



testing_losses = torch.load(render_losses_path)
avg_losses = np.array([x[0].item() for x in testing_losses])
part_sizes = np.array([x[1] for x in testing_losses])
total_losses = avg_losses * part_sizes
avg_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][0])
total_sorted = sorted(enumerate(zip(avg_losses, part_sizes, total_losses)),key=lambda x: x[1][2])
total_filtered = [x for x in total_sorted if x[1][1] > 20]


recompute_renderings = True
if os.path.exists('render_results.pt') and not force_rendering:
    render_results = torch.load('render_results.pt')
    renders = render_results['renders']
    gts = render_results['gts']
    if len(renders) >= rows*cols:
        recompute_renderings = False

if recompute_renderings:
    with open(f360seg_index_path,'r') as f:
        index = json.load(f)
    data =  ZipFile(f360seg_zip_path,'r')
    parts_list = [index['template'].format(*x) for x in index['test']]

    model = BRepFaceAutoencoder(64,1024,4)
    ckpt = torch.load(model_checkpoint_path)
    model.load_state_dict(ckpt['state_dict'])

    def predict(face_codes, model, N=grid_density):
        n_faces = face_codes.shape[0]
        line = torch.linspace(-0.1,1.1,N)
        grid = torch.cartesian_prod(line, line)
        grids = grid.repeat(n_faces,1)
        indices = torch.arange(n_faces).repeat_interleave(N*N, dim=0)
        with torch.no_grad():
            indexed_codes = face_codes[indices]
            uv_codes = torch.cat([grids, indexed_codes],dim=1)
            preds = model.decoder(uv_codes)
        return preds

    codes = torch.load(computed_f360seg_codes_path)

    gts = []
    renders = []

    poses = np.load(test_poses_path)
    zooms = np.load(test_zooms_path)

    for k in tqdm(range(rows*cols)):
        i = total_filtered[k][0]
        N = grid_density
        preds = predict(codes[parts_list[i]]['x'], model, N)
        V, F, C = preds_to_mesh(preds, N)
        part = Part(data.open(parts_list[i]).read().decode('utf-8'))
        pose = poses[i]
        zoom = zooms[i]
        #pose, zoom = find_best_angle_from_part(part)
        ground_truth = render_part(part, pose, zoom)
        rendering = render_mesh(V,F,C,pose,zoom)
        gts.append(ground_truth)
        renders.append(rendering)

    render_results = {'gts':gts,'renders':renders}
    torch.save(render_results, 'render_results.pt')



M = rows*cols
s = size
fig, axes = plt.subplots(int(M/cols), cols*2, figsize=(2*cols*s, s*int(M/cols)),gridspec_kw = {'wspace':0, 'hspace':0}, dpi=300)
for i in range(M):
    row = int(i / cols)
    col = i % cols
    axes[row,2*col].imshow(gts[i])
    axes[row,2*col].axis('off')
    axes[row,2*col+1].imshow(renders[i])
    axes[row,2*col+1].axis('off')
fig.savefig(figure_out_path)

  0%|          | 0/24 [00:00<?, ?it/s]

IndexError: index 2711 is out of bounds for axis 0 with size 1548

In [20]:
poses.shape

(1548, 4, 4)

## MFCAD Figure

In [ ]:
from precoded_training import DictDatamodule, CodePredictor, load_models
from pylab import cm
import meshplot as mp

with open('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.json','r') as f:
    index = json.load(f)
    coded_set = torch.load('/projects/grail/benjonesnb/cadlab/siggasia2022/precoded/mfcad_coded.pt')

datamodule = DictDatamodule(
                    index, 
                    coded_set,  
                    seed=0, 
                    batch_size=512
                )

test_ds = datamodule.ds_test

all_models = load_models('/projects/grail/benjonesnb/cadlab/siggasia2022/tensorboard/mfcad/')
our_models = all_models['mp2']

# Compute a "difficulty" scale for test set examples (this is also used in comparison figs...)
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# Setup Plotting Code (Replace with Other Version)
num_labels = max([i for j in index['test_labels'] for i in j]) + 1
cmap = cm.get_cmap('tab20', num_labels)
zf = ZipFile('/projects/grail/benjonesnb/cadlab/siggasia2022/mfcad.zip')
test_keys = [index['template'].format(*x) for x in index['test']]
def plot_part(V, F, E2T, FC=None):
    plot = mp.plot(V, F, c = FC)
    E = np.concatenate([
        F[E2T[:,0]>=0][:,[2,0]], 
        F[E2T[:,1]>=0][:,[0,1]],
        F[E2T[:,2]>=0][:,[1,2]]
    ], axis=0)
    plot.add_edges(V, E, shading={'line_width':0.5})
def plot_preds(i, test_ds, test_keys=test_keys, zf=zf, models=our_models, colormap=cmap, just_gt=False):
    data = test_ds[i]
    with zf.open(test_keys[i],'r') as f:
        p = Part(f.read().decode('utf-8'))
    target = data.y.numpy()
    target_c = colormap(target[p.mesh_topology.face_to_topology])[:,:3]
    plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology,  target_c)
    if just_gt:
        return
    model_preds = []
    for ts, model in models:
        with torch.no_grad():
            preds = model(data).argmax(dim=1).numpy()
            model_preds.append(preds)   
    for preds in model_preds:
        pred_c = colormap(preds[p.mesh_topology.face_to_topology])[:,:3]
        plot_part(p.mesh.V, p.mesh.F, p.mesh_topology.edge_to_topology, pred_c)


# Metrics used to help find the examples
local_auc = accs.sum(axis = 1) - accs[:,0] / 2 - accs[:,-1] / 2
complexity = np.array([len(test_ds[i].y) for i in range(len(test_ds))])
sorted_by_auc = sorted(enumerate(zip(local_auc, complexity)), key=lambda x:x[1][0])
filtered_examples = [x for x in sorted_by_auc if x[1][1] > 20]

gallery_examples = [646, 965, 673, 417, 888, 1090]

for i in gallery_examples:
    plot_preds(i, test_ds, test_keys, zf, our_models, cmap, just_gt=True)


## Limitations Plot

In [239]:
import altair as alt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import altair_saver

accs = pd.read_parquet('df_face_acc.parquet')
accs['one'] = 1
iscorr = np.vectorize(lambda x: 'correct' if x == 1 else 'incorrect')
accs['correct'] = iscorr(accs.label)
accs['logmse'] = np.log(accs.mse)


alt.data_transformers.disable_max_rows()

lim_plot = alt.Chart(accs[accs.mse < 1]).mark_bar().encode(
    x=alt.X('logmse',bin=alt.Bin(maxbins=50)),
    y=alt.Y('sum(one)', stack='normalize', axis=alt.Axis(format='%')),
    color=alt.Color('label:N')
).facet(row='train_size')


In [240]:
altair_saver.save(lim_plot, 'lim_plot.pdf')

WARN row encoding should be discrete (ordinal / nominal / binned).


## Classification Comparisons

In [ ]:
import numpy as np
import torch
import json
from precoded_training import DictDatamodule, CodePredictor, load_models
from rendering import find_best_angle_from_part, render_part2
from tqdm.notebook import tqdm
from pspy import Part
from zipfile import ZipFile
from matplotlib import pyplot as plt


# Setup Local Paths

repbrep = '/home/ben/Documents/research/repbrep/'
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')

f360seg_index_path =  os.path.join(repbrep, 'datasets', 'fusion360seg.json')
f360seg_zip_path = os.path.join(repbrep, 'datasets', 'fusion360seg.zip')
f360seg_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'f360seg')
f360seg_coded_path = os.path.join(model_dir, 'fusion360seg_coded.pt')

mfcad_index_path =  os.path.join(repbrep, 'datasets', 'mfcad.json')
mfcad_zip_path = os.path.join(repbrep, 'datasets', 'mfcad.zip')
mfcad_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'mfcad')
mfcad_coded_path = os.path.join(model_dir, 'mfcad_coded.pt')

f360seg_uvnet_results_path = os.path.join(repbrep, 'results', 'Fusion360Seg_UV-Net.json')
f360seg_brepnet_results_path = os.path.join(repbrep, 'results', 'Fusion360Seg_BRepNet.json')

#data_root = '/media/ben/Data/cad'
#f360seg_index_path =  data_root + '/fusion360seg.json'
#f360seg_zip_path = data_root + '/fusion360seg.zip'

#repbrep_data_path = '/home/ben/Documents/research/repbrep/data'

#f360_seg_coded_path = repbrep_data_path + '/precoded/fusion360seg_coded.pt'

#f360_seg_uvnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_UV-Net.json'
#f360seg_brepnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_BRepNet.json'

#experiments_dir = '/home/ben/Documents/research/repbrep/data/training/experiments'
#f360seg_tb_dir = experiments_dir +'/f360seg/'



# Load UV-Net Results
with open(f360seg_uvnet_results_path,'r') as f:
    uvnet360 = json.load(f)

labels = uvnet360['labels']
logits = uvnet360['logits']['100'][0]
uv_preds = [np.array(l).argmax() for l in logits]
part_accs_uv = []
offset = 0
part_preds_uv = []
for part_labels in labels:
    part_preds = uv_preds[offset:offset+len(part_labels)]
    part_preds_uv.append(part_preds)
    part_accs_uv.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
    offset += len(part_labels)
part_accs_uv = np.array(part_accs_uv)

# Load BRepNet Results
with open(f360seg_brepnet_results_path,'r') as f:
    brep360 = json.load(f)

labels = brep360['labels']
logits = brep360['logits']['100'][0]
preds = [np.array(l).argmax() for ll in logits for l in ll]
part_accs_brep = []
part_preds_brep = []
offset = 0
for i, part_labels in enumerate(labels):
    part_preds = preds[offset:offset+len(part_labels)]
    part_preds_brep.append(part_preds)
    part_accs_brep.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
    offset += len(part_labels)
part_accs_brep = np.array(part_accs_brep)


# Load Our Trained Models
all_models = load_models(f360seg_tb_dir)
our_models = all_models['mp2']

# Select Our Model
model = our_models[1][1]

# Load Fusion 360 Dataset
with open(f360seg_index_path,'r') as f:
    index = json.load(f)
    coded_set = torch.load(f360seg_coded_path)



datamodule = DictDatamodule(
                    index, 
                    coded_set, 
                    train_size=23266, 
                    seed=0, 
                    batch_size=512
                )

test_ds = datamodule.ds_test


# Compute Our Model's Accuracy at different train sizes
accs = np.zeros((len(test_ds), len(our_models)))
for i in tqdm(range(len(test_ds))):
    data = test_ds[i]
    for j, (ts, m) in enumerate(our_models):
        with torch.no_grad():
            preds = m(data).argmax(dim=1).numpy()
        targets = data.y.numpy()
        acc = (preds.flatten() == targets.flatten()).sum() / len(targets.flatten())
        accs[i,j] = acc

# We just take the accuracies for the size we used
part_accs_ours = accs[:,1]


# Compute the total accuracy lift over baselines for each test example
lift = (part_accs_ours - part_accs_brep) + (part_accs_ours - part_accs_uv)

# Compute the number of faces in each test example as a complexity metric
complexity = np.array([len(test_ds[i].y) for i in range(len(test_ds))])

# Sort by lift and other keys, and filter to parts with more than 25 faces, and baselines with at least 10% accuracy
nuanced_wins = [x for x in sorted(enumerate(zip(lift,complexity, part_accs_ours, part_accs_uv, part_accs_brep)), reverse=True, key = lambda x : x[1]) 
 if x[1][1] > 25 and x[1][3] > .1 and x[1][4] > .1]

# Here is where we could try selecting random models.


data =  ZipFile(f360seg_zip_path,'r')
parts_list = [index['template'].format(*x) for x in index['test']]

s=5
M = 10

selection = nuanced_wins[:M]
fig, axes = plt.subplots(M,4, figsize=(4*s,M*s),gridspec_kw={'wspace':0,'hspace':0},dpi=300)
for k, sel in enumerate(tqdm(selection)):
    i = sel[0]
    part = Part(data.open(parts_list[i]).read().decode('utf-8'))
    pose, zoom = find_best_angle_from_part(part, cubify=True)

    gt = np.array(labels[i])
    pdata = test_ds[i]
    our_preds = model(pdata).argmax(dim=1).numpy().astype(int)
    uv_preds = np.array(part_preds_uv[i]).astype(int)
    brep_preds = np.array(part_preds_brep[i]).astype(int)

    gt_im = render_part2(part, pose, zoom, face_labels=gt, max_labels=8)
    our_im = render_part2(part, pose, zoom, face_labels=our_preds, max_labels=8)
    uv_im = render_part2(part, pose, zoom, face_labels=uv_preds, max_labels=8)
    brep_im = render_part2(part, pose, zoom, face_labels=brep_preds, max_labels=8)
    axes[k,0].imshow(gt_im)
    axes[k,1].imshow(our_im)
    axes[k,2].imshow(uv_im)
    axes[k,3].imshow(brep_im)
    for ax in axes[k]:
        ax.axis('off')

In [ ]:
import os
import json
import torch
from tqdm.notebook import tqdm
from precoded_training import DictDatamodule, CodePredictor, load_models

repbrep = '/home/ben/Documents/research/repbrep/'
model_dir = os.path.join(repbrep, 'models', 'BRepFaceAutoencoder_64_1024_4')
checkpoint_path = os.path.join(model_dir, 'BRepFaceAutoencoder_64_1024_4.ckpt')

f360seg_index_path =  os.path.join(repbrep, 'datasets', 'fusion360seg.json')
f360seg_zip_path = os.path.join(repbrep, 'datasets', 'fusion360seg.zip')
f360seg_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'f360seg')
f360seg_coded_path = os.path.join(model_dir, 'fusion360seg_coded.pt')

mfcad_index_path =  os.path.join(repbrep, 'datasets', 'mfcad.json')
mfcad_zip_path = os.path.join(repbrep, 'datasets', 'mfcad.zip')
mfcad_tb_dir = os.path.join(repbrep, 'training_logs', 'ours', 'mfcad')
mfcad_coded_path = os.path.join(model_dir, 'mfcad_coded.pt')


our_models_all_seeds_f360 = [load_models(f360seg_tb_dir, s)['mp2'] for s in range(10)]
our_models_all_seeds_mfcad = [load_models(mfcad_tb_dir, s)['mp2'] for s in range(10)]


def load_testset(index_path, coded_path):
    with open(index_path, 'r') as f:
        index = json.load(f)
    coded_set = torch.load(coded_path)
    datamodule = DictDatamodule(
        index,
        coded_set,
        train_size=1000,
        seed=0,
        batch_size=100000
    )
    test_ds = datamodule.ds_test
    return test_ds

f360_test = load_testset(f360seg_index_path, f360seg_coded_path)
mfcad_test = load_testset(mfcad_index_path, mfcad_coded_path)


labels = []
records = []
for seed in tqdm(range(10), 'Seed'):
    for model_name, model_set, test_ds in tqdm([
        ('Fusion360Seg', our_models_all_seeds_f360, f360_test), 
        ('MFCAD', our_models_all_seeds_mfcad, mfcad_test)], 'dataset', leave=False):
        for train_size, model in tqdm(model_set[seed], 'train_size', leave=False):
            for i in tqdm(range(len(test_ds)), 'examples', leave=False):
                data = test_ds[i]
                with torch.no_grad():
                    pred_logits = model(data).numpy().tolist()
                iter_labels = data.y.numpy().tolist()
                records.append({
                    'dataset': model_name,
                    'model': 'Ours',
                    'test_idx': i,
                    'seed': seed,
                    'train_size': train_size,
                    'logits': pred_logits,
                    'labels': iter_labels
                })

In [11]:
import json
import numpy as np
from tqdm.notebook import tqdm

data_root = '/media/ben/Data/cad'
f360seg_index_path =  data_root + '/fusion360seg.json'
f360seg_zip_path = data_root + '/fusion360seg.zip'

repbrep_data_path = '/home/ben/Documents/research/repbrep/data'

f360_seg_coded_path = repbrep_data_path + '/precoded/fusion360seg_coded.pt'

f360seg_uvnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_UV-Net.json'
f360seg_brepnet_results_path = repbrep_data_path + '/baselines/Fusion360Seg_BRepNet.json'

mfcad_uvnet_results_path = repbrep_data_path + '/baselines/MFCAD_UV-Net.json'
mfcad_brepnet_results_path = repbrep_data_path + '/baselines/MFCAD_BRepNet.json'

experiments_dir = '/home/ben/Documents/research/repbrep/data/training/experiments'
f360seg_tb_dir = experiments_dir +'/f360seg/'

def load_json(path):
    with open(path,'r') as f:
        return json.load(f)

def load_uvnet_results(uvnet360, train_size, seed):
    labels = uvnet360['labels']
    logits = uvnet360['logits'][str(train_size)][seed]
    uv_preds = [np.array(l).argmax() for l in logits]
    part_accs_uv = []
    offset = 0
    part_preds_uv = []
    for part_labels in labels:
        part_preds = uv_preds[offset:offset+len(part_labels)]
        part_preds_uv.append(part_preds)
        part_accs_uv.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
        offset += len(part_labels)
    part_accs_uv = np.array(part_accs_uv)
    return part_preds_uv, part_accs_uv

def load_all_uv_net_results(uvnet360, seed):
    sizes = [int(x) for x in uvnet360['logits'].keys()]
    results = {}
    for s in sizes:
        preds, accs = load_uvnet_results(uvnet360, s, seed)
        results[s] = {'preds':preds,'accs':accs}
    return results


def load_brepnet_results(brep360, size, seed):
    labels = brep360['labels']
    logits = brep360['logits'][str(size)][seed]
    preds = [np.array(l).argmax() for ll in logits for l in ll]
    part_accs_brep = []
    part_preds_brep = []
    offset = 0
    for i, part_labels in enumerate(labels):
        part_preds = preds[offset:offset+len(part_labels)]
        part_preds_brep.append(part_preds)
        part_accs_brep.append((np.array(part_preds) == np.array(part_labels)).sum() / len(part_labels))
        offset += len(part_labels)
    part_accs_brep = np.array(part_accs_brep)
    return part_preds_brep, part_accs_brep

#res = load_all_uv_net_results(uvnet360, 0)